In [0]:
dbutils.widgets.text("Environment", "dev", "Set the current environment/catalog name")
dbutils.widgets.text("RunType", "once", "Set once to run as a batch")
dbutils.widgets.text("ProcessingTime", "5 seconds", "Set the microbatch interval")

In [0]:
env = dbutils.widgets.get("Environment")
once = True if dbutils.widgets.get("RunType")=="once" else False
processing_time = dbutils.widgets.get("ProcessingTime")

if once:
    print(f"Starting sbit in batch mode.")
else:
    print(f"Starting sbit in stream mode with {processing_time} microbatch.")

In [0]:
spark.conf.set("spark.sql.shuffle.partitions", sc.defaultParallelism)
spark.conf.set("spark.databricks.delta.optimizeWrite.enabled", True)
spark.conf.set("spark.databricks.delta.autoCompact.enabled", True)
spark.conf.set("spark.sql.streaming.stateStore.providerClass", "com.databricks.sql.streaming.state.RocksDBStateStoreProvider")

In [0]:
%run "./02_setup"

In [0]:
%run "./03_history_loader"

In [0]:
SH = SetupHelper(env)
HL = HistoryLoader(env)

In [0]:
setup_required = spark.sql(f"SHOW DATABASES IN {SH.catalog}").filter(f"databaseName == '{SH.db_name}'").count() != 1
if setup_required:
    SH.setup()
    SH.validate()
    HL.load_history()
else:
    spark.sql(f"USE {SH.catalog}.{SH.db_name}")

In [0]:
%run "./04_bronze"

In [0]:
bronze = Bronze(env)

In [0]:
bronze.consume(once, processing_time)

In [0]:
%run "./05_silver"

In [0]:
silver = Silver(env)

In [0]:
silver.upsert(once, processing_time)

In [0]:
%run "./06_gold"

In [0]:
gold = Gold(env)

In [0]:
gold.upsert(once, processing_time)

In [0]:
%sql

select * from fitbit_dev_catalog.fitbit_db.calories_min_bz order by date desc limit 5

In [0]:
%sql
select * from fitbit_dev_catalog.fitbit_db.calories_min_bz order by date desc limit 5

In [0]:
%sql

select * from fitbit_dev_catalog.fitbit_db.activity_daily_gold order by date desc limit 5